In [1]:
from pprint import pprint

from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "/home/zcwang/data/model/google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# print(tokenizer.special_tokens_map.items())
# print special tokens <special_name, cotent, id>
for k, v in tokenizer.special_tokens_map.items():
    print(k, v, tokenizer.convert_tokens_to_ids(v))

model.cuda("cuda:0")

/home/zcwang/.conda/envs/trace/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zcwang/.conda/envs/trace/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:48<00:00, 24.07s/it]


bos_token <bos> 2
eos_token <eos> 1
unk_token <unk> 3
pad_token <pad> 0
additional_special_tokens ['<start_of_turn>', '<end_of_turn>'] [106, 107]


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

In [25]:
# batched input
input_text = ["The dog is happy", "So sad"]

# left padding
tokenizer.padding_side = "left"
# tokenizer.pad_token = tokenizer.eos_token

# tokenize input
input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# move input to cuda
input_ids = {k: v.cuda() for k, v in input_ids.items()}
print(input_ids)

# batched generation and don't do sample but greedy decoding
generate_config = {
    "max_length": 50,
    "num_return_sequences": 3,
    "do_sample": True,
}
batched_output = model.generate(input_ids["input_ids"], **generate_config)
print(batched_output.shape)

decodings = tokenizer.batch_decode(batched_output, skip_special_tokens=True)
for output in decodings:
    print('#' * 30)
    print(output)

{'input_ids': tensor([[   2,  651, 5929,  603, 4915],
        [   0,    0,    2, 2339, 9270]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1]], device='cuda:0')}
torch.Size([6, 50])
##############################
The dog is happy at being out in nature and enjoys sniffing and chasing after leaves

What is the main idea of the passage?

The main idea is that the dog is happy and enjoys being out in nature.
##############################
The dog is happy running alone, but when he is leash-trained, he pulls the rope and runs away.

The solution to this problem could be that the leash restricts the dog's ability to move forward.  Sure, the leash
##############################
The dog is happy.

It wagged its tail happily and licked its own face with a satisfied grin.

The word "happy" was clearly expressed on its face, as its lips curled up in a genuine smile.

The dog'
##############################
So sad, the woman who used to work with me at the b

In [18]:
tokenizer.padding_side = "right"
generate_config = {
    "max_length": 50,
    "num_return_sequences": 2,
    "do_sample": True,
}

one_by_one_output = []
# generate one by one
for i in range(len(input_text)):
    input_id = tokenizer(input_text[i], return_tensors="pt", padding=False, truncation=True)
    input_id = {k: v.cuda() for k, v in input_id.items()}
    output = model.generate(input_id["input_ids"], **generate_config)
    print(output.shape)
    # one_by_one_output.append(output[0])
    one_by_one_output.append(output)
    # print(tokenizer.decode(output[0], skip_special_tokens=True))


decodings = tokenizer.batch_decode(one_by_one_output, skip_special_tokens=True)
for output in decodings:
    print('#' * 30)
    print(output)

torch.Size([2, 50])
torch.Size([2, 50])


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [22]:
import torch
# [2, 50] cat [5] -> [2, 55]
ones = torch.ones(2, 50)
zeros = torch.zeros(5)
zeros = zeros.repeat(2, 1)
print(torch.cat([ones, zeros], dim=1).shape)

torch.Size([2, 55])


In [27]:
# a example for torch.mode
a = torch.tensor([[2, 2, 3], [4, 5, 6]])
print(torch.mode(a, dim=1))
print(torch.mode(a, dim=0))
print(torch.mode(a, dim=-1))
print(torch.mode(a, dim=-2))

torch.return_types.mode(
values=tensor([2, 4]),
indices=tensor([1, 0]))
torch.return_types.mode(
values=tensor([2, 2, 3]),
indices=tensor([0, 0, 0]))
torch.return_types.mode(
values=tensor([2, 4]),
indices=tensor([1, 0]))
torch.return_types.mode(
values=tensor([2, 2, 3]),
indices=tensor([0, 0, 0]))
